In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, accuracy_score
from sklearn.linear_model import LinearRegression
import pickle
import tensorflow as tf
from tensorflow.keras import layers
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier


En este notebook programaremos diferentes modelos para poder determinar la probabilidad de atraso de un vuelo. Para esto, asumiremos que un vuelo se considerará atrasado solamente si el atraso no es de la categoría menor. Es decir, si su hora de salida es mayor a 15 minutos de la hora calendarizada inicialmente.

## Preprocesamiento de datos

Partimos leyenddo los datos y definiendo la variable objetivo.

In [2]:
!gdown 1zqZqK8IwB78lL03b_d5agsPZH6lxn1sH

Downloading...
From: https://drive.google.com/uc?id=1zqZqK8IwB78lL03b_d5agsPZH6lxn1sH
To: /content/synthetic_features.csv
100% 9.98M/9.98M [00:00<00:00, 74.8MB/s]


In [3]:
df = pd.read_csv('synthetic_features.csv')
df['Fecha-I'] = pd.to_datetime(df['Fecha-I'], format='%Y-%m-%d %H:%M:%S')
df['Fecha-O'] = pd.to_datetime(df['Fecha-O'], format='%Y-%m-%d %H:%M:%S')
df['Atrasado'] = (df['Diferencia en minutos'] > 0).astype(int)
df.head()

<ipython-input-3-26cd87a1a57f>:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('synthetic_features.csv')


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,OPERA,SIGLAORI,SIGLADES,Count,Temporada Alta,Temporada alta,Diferencia en minutos,Atraso menor,Periodo día,Atrasado
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,American Airlines,Santiago,Miami,1,1,1,3.0,1,Noche,1
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,American Airlines,Santiago,Miami,1,1,1,9.0,1,Noche,1
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,American Airlines,Santiago,Miami,1,1,1,9.0,1,Noche,1
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,American Airlines,Santiago,Miami,1,1,1,3.0,1,Noche,1
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,American Airlines,Santiago,Miami,1,1,1,-2.0,0,Noche,0


No usaremos todas las columnas ya que algunas no entregan información relevante o son redundantes. Por tanto, seleccionamos las que sí utilizaremos. Por ejemplo, con el riesgo de perder un poco de información, tomamos los siguientes supuestos:

1. la columna del número de vuelo programado es redundante, ya que si tomamos los vuelos que son de X aerolinea, van al destino W, y operan en el período del día Z, entonces ya podemos saber cual es número de vuelo.
2. La columna de ano no es relevante ya que todos los vuelos son del 2017.
3. SIGLAORI no es relevante ya que todos los vuelos salen de SCL.


Por otro lado, solo es válido usar las columnas que no terminan en "-O", ya que estas nos entregan información posterior a la operación.

In [4]:
df = df[['Fecha-I', 'Des-I', 'Emp-I', 'TIPOVUELO', 'Periodo día', 'Atrasado', 'Temporada alta']]
df.head()

,Fecha-I,Des-I,Emp-I,TIPOVUELO,Periodo día,Atrasado,Temporada alta
0,2017-01-01 23:30:00,KMIA,AAL,I,Noche,1,1
1,2017-01-02 23:30:00,KMIA,AAL,I,Noche,1,1
2,2017-01-03 23:30:00,KMIA,AAL,I,Noche,1,1
3,2017-01-04 23:30:00,KMIA,AAL,I,Noche,1,1
4,2017-01-05 23:30:00,KMIA,AAL,I,Noche,0,1


Agregamos columnas con día de semana, día del mes y mes.

In [5]:
df['DIANOM-I'] = df['Fecha-I'].apply(lambda x: x.weekday())
df['DIA-I'] = df['Fecha-I'].apply(lambda x: x.day)
df['MES-I'] = df['Fecha-I'].apply(lambda x: x.month)
df = df.drop(['Fecha-I'], axis=1)
df.head()

,Des-I,Emp-I,TIPOVUELO,Periodo día,Atrasado,Temporada alta,DIANOM-I,DIA-I,MES-I
0,KMIA,AAL,I,Noche,1,1,6,1,1
1,KMIA,AAL,I,Noche,1,1,0,2,1
2,KMIA,AAL,I,Noche,1,1,1,3,1
3,KMIA,AAL,I,Noche,1,1,2,4,1
4,KMIA,AAL,I,Noche,0,1,3,5,1


Las variables categóricas las convertiremos a binarias a través de la función pd.get_dummies(). Esta función es un OneHotEncoder.

In [6]:
data = pd.get_dummies(df)
data = pd.get_dummies(data, columns=['DIANOM-I', 'DIA-I','MES-I'])
data

,Atrasado,Temporada alta,Des-I_CYYZ,Des-I_EGLL,Des-I_EGYP,Des-I_KATL,Des-I_KDFW,Des-I_KIAH,Des-I_KJFK,Des-I_KLAX,...,MES-I_3,MES-I_4,MES-I_5,MES-I_6,MES-I_7,MES-I_8,MES-I_9,MES-I_10,MES-I_11,MES-I_12
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68202,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68203,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68204,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Modelos

Partiremos probando modelos simples.  A medida de que logremos ir entendiendo mejor los datos y la naturaleza del modelo que se necesita los iremos complejizando.

### Métrica de rendimiento
Ya que se nos encontrar la probabilidad de atraso de un vuelo, utilizaremos la métrica de logloss. ESta métrica es adecuada para predecir probabilidades de eventos, tal como el atraso de aviones en nuestro caso. Logloss penaliza fuertemente las predicciones incorrectas, asignando una pérdida mayor cuando el modelo está confiado de una clase incorrecta. Al optimizar este valor, se fomenta que el modelo genere probabilidades más calibradas y más cercanas a la realidad. Para esta métrica, mientras más bajo el valor, mejor.

Por otro lado, también le pondremos atención al accuracy. Esta es una métrica intuitiva y fácil de interpretar. Representa la proporción de predicciones correctas realizadas por el modelo en relación con el total de muestras. Al medir la capacidad del modelo para clasificar correctamente las muestras entre las que tienen una alta probabilidad de error y las que no, el accuracy proporciona una medida general de la precisión del modelo.


### Regresión lineal

In [16]:
X = data.drop('Atrasado', axis=1)
y = data['Atrasado']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)

In [31]:
# creating a regression model
model = LinearRegression()

# fitting the model
model.fit(X_train, y_train)

# making predictions
predictions = model.predict(X_test)

In [32]:
print(f'log loss: {log_loss(y_test, predictions)}')
predictions = [0 if i<0.5 else 1 for i in predictions]
print(f'Accuracy : {accuracy_score(y_test, predictions)}')

log loss: 0.5983974922451969
Accuracy : 0.6934805981819959


Vemos que el valor es muy malo, ya que apenas supera en 0.02 unidades al baseline de 0.66. También podemos fijarnos en el score de una regresión lineal es el valor $R^2$. Este parámetro se puede interpretar como que porcentaje de los datos es capaz de predecir la regresión. El valor va entre 0 y 1. Un valor cercano a 0 significa que el modelo no se ajusta bien a los datos.

### Random Forest
Ahora pasamos a correr un modelo de Random Forest. Probaremos su rendiemiento para varios parámetros de número de estimadores, criterios de clasificación y profundidad.

In [ ]:
X = data.drop('Atrasado', axis=1)
y = data['Atrasado']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=11)

In [ ]:
n_estimators = [10, 20, 50]
criterio = ['gini', 'entropy']
depth = [10, 20, 50]

In [ ]:
for n in n_estimators:
    for c in criterio:
        for d in depth:
            model = RandomForestClassifier(max_depth=d, random_state=0, n_estimators=n, criterion = c)
            model.fit(X_train, y_train)

            predictions = model.predict(X_test)
            log_loss_ =  log_loss(y_test, predictions)
            predictions = [0 if i<0.5 else 1 for i in predictions]
            score = accuracy_score(y_test, predictions)


            print(f'Estimators:{n}, Criterion:{c}, depth: {d}, logloss:{log_loss_}, score:{score}')

Estimators:10, Criterion:gini, depth: 10, logloss:11.493736602677616, score:0.6811162154237123
Estimators:10, Criterion:gini, depth: 20, logloss:10.996995802377988, score:0.6948978594467794
Estimators:10, Criterion:gini, depth: 50, logloss:12.193049005936317, score:0.661714397419607
Estimators:10, Criterion:entropy, depth: 10, logloss:11.500782571476195, score:0.6809207311113283
Estimators:10, Criterion:entropy, depth: 20, logloss:10.951197005187241, score:0.696168507477275
Estimators:10, Criterion:entropy, depth: 50, logloss:12.214186912332046, score:0.6611279444824553
Estimators:20, Criterion:gini, depth: 10, logloss:11.474360188481533, score:0.6816537972827681
Estimators:20, Criterion:gini, depth: 20, logloss:10.967050434984037, score:0.6957286677744111
Estimators:20, Criterion:gini, depth: 50, logloss:11.986954418577959, score:0.6674323135568371
Estimators:20, Criterion:entropy, depth: 10, logloss:11.446176313287227, score:0.6824357345323038
Estimators:20, Criterion:entropy, depth:

Notamos que los resultados obtenidos tampoco son buenos. Tuvimos apenas un logloss míinimo de 10 aproximadamente. Esto es esperable debido a que los random forest predicen directmanete en clases, pero no estiman bien probabilidades. Pasamos ahora a probar los modelos de Support Vector Machines.

### Support Vector Machines

Por temas de tiempo se decide usar un sample del dataset original de 7.000 filas para probar diferentes configuraciones de manera sencilla. Luego la mejor de ellas se ajustará con el dataset completo.

In [ ]:
n = 7000
data_ = data.sample(n)
X = data_.drop('Atrasado', axis=1)
y = data_['Atrasado']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=11)

In [ ]:
C = [0.01, 1, 100]
kernels = ['linear', 'rbf', 'poly']

In [ ]:
for kernel in kernels:
  for c in C:
      model = SVR(kernel=kernel, C=c)
      model.fit(X_train, y_train)

      predictions = model.predict(X_test)
      log_loss_ =  log_loss(y_test, predictions)
      predictions = [1 if i>0.5 else 0 for i in predictions]
      score = accuracy_score(y_test, predictions)

      print(f'Kernel: {kernel}, C:{c}, logloss: {log_loss_}, score:{score}')

Kernel: linear, C:0.01, logloss: 11.237610156651415, score:0.6882222222222222
Kernel: linear, C:1, logloss: 11.181542251379454, score:0.6897777777777778
Kernel: linear, C:100, logloss: 11.181542251379454, score:0.6897777777777778
Kernel: rbf, C:0.01, logloss: 11.750231004852193, score:0.674
Kernel: rbf, C:1, logloss: 10.965280331044752, score:0.6957777777777778
Kernel: rbf, C:100, logloss: 13.528384572048639, score:0.6246666666666667
Kernel: poly, C:0.01, logloss: 11.750231004852193, score:0.674
Kernel: poly, C:1, logloss: 11.686153398827095, score:0.6757777777777778


In [ ]:
X = data.drop('Atrasado', axis=1)
y = data['Atrasado']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=11)

In [ ]:
best_c = 1
model = SVR(kernel='rbf', C=best_c)
model.fit(X_train.values, y_train.values)
predictions = model.predict(X_test)
log_loss_ =  log_loss(y_test, predictions)
predictions = [1 if i>0.5 else 0 for i in predictions]
score = accuracy_score(y_test, predictions)
print(f'logloss: {log_loss_}, score:{score}')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


logloss: 1.1433012768929627, score:0.7080441794545987


### Red Neuronal

In [ ]:
X = data.drop('Atrasado', axis=1)
y = data['Atrasado']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=11)

In [ ]:
# Create a sequential model
model = Sequential()
# Add a dense layer with 16 units and 'relu' activation function
model.add(Dense(16, activation='relu', input_dim=150))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
# Add a final dense layer with 1 unit and 'sigmoid' activation function
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=32)
# Predict the binary values for the test data
predictions = model.predict(X_test)

Epoch 1/25
1492/1492 [==============================] - 5s 2ms/step - loss: 0.6441 - accuracy: 0.6633
Epoch 2/25
1492/1492 [==============================] - 4s 2ms/step - loss: 0.6299 - accuracy: 0.6719
Epoch 3/25
1492/1492 [==============================] - 4s 3ms/step - loss: 0.6173 - accuracy: 0.6780
Epoch 4/25
1492/1492 [==============================] - 2s 1ms/step - loss: 0.6116 - accuracy: 0.6800
Epoch 5/25
1492/1492 [==============================] - 2s 1ms/step - loss: 0.6076 - accuracy: 0.6832
Epoch 6/25
1492/1492 [==============================] - 2s 1ms/step - loss: 0.6043 - accuracy: 0.6834
Epoch 7/25
1492/1492 [==============================] - 3s 2ms/step - loss: 0.6028 - accuracy: 0.6847
Epoch 8/25
1492/1492 [==============================] - 2s 1ms/step - loss: 0.6002 - accuracy: 0.6870
Epoch 9/25
1492/1492 [==============================] - 2s 1ms/step - loss: 0.5983 - accuracy: 0.6865
Epoch 10/25
1492/1492 [==============================] - 2s 1ms/step - loss: 0.596

In [ ]:
print(f'log loss: {log_loss(y_test, predictions)}')
predictions = [0 if i<0.5 else 1 for i in predictions]
print(f'Accuracy : {accuracy_score(y_test, predictions)}')

log loss: 0.5870208227449433
Accuracy : 0.6906949467305249


### Mejor modelo

El modelo que mejor optimizó la métrica logloss fue la red neuronal. Sin embargo, es importante notar que el punteje de logloss de la regresión lineal fue de apenas 0.01 unidades de diferencia. En términos prácticos, se puede decir que ambos modelos tienen el mismo poder de predicción. Sin embargo, el modelo de la regresión lineal es mucho más sencillo y liviano en término de complejidad y espacio. Esto puede favorecer el rendimiento de nuestra API. Por tanto, elejimos el modelo de regresión lineal como el mejor para serializar. Usaremos la librería Pickle para exportarlo y luego instanciarlo en la API.

In [33]:
model = LinearRegression()
model.fit(X_train, y_train)
filename = 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))